# 第七章：使用示例（少样本提示）

本章将学习如何通过提供示例来改善GPT的响应质量和格式。这种技术被称为"**少样本提示**"（Few-Shot Prompting），是提示工程中最有效的技术之一。

## 🎯 学习目标

通过本章的学习，您将掌握：
- 理解少样本提示的核心原理和应用场景
- 学会设计有效的示例来引导模型行为
- 掌握不同类型示例的使用技巧
- 能够针对特定任务构建高质量的少样本提示

## 💡 核心概念

**少样本提示**是一种通过提供少量示例来教导模型如何执行特定任务的技术。相比于详细的文字描述，示例往往能更直观、更准确地传达我们的期望。

### 为什么少样本提示如此有效？

1. **直观性**：示例比抽象描述更容易理解
2. **格式控制**：通过示例可以精确控制输出格式
3. **一致性**：模型倾向于模仿示例的风格和结构
4. **减少歧义**：示例消除了指令中的模糊性

## 设置

运行以下设置单元格来加载您的API密钥并建立`get_completion`辅助函数。

In [1]:
# 安装OpenAI库
%pip install openai==1.61.0

# 🔧 OpenAI环境自动配置
# 此设置会自动从环境变量或IPython存储中加载配置

# 安装OpenAI库
%pip install openai==1.61.0

# 导入Python内置的正则表达式库
import re

# 🚀 使用统一配置管理系统
from config import setup_notebook_environment, print_config_info

# 自动设置OpenAI客户端和get_completion函数
# 优先级：环境变量 > IPython存储 > 默认值
try:
    client, get_completion = setup_notebook_environment()
    print("✅ 使用统一配置管理成功！")
except Exception as e:
    print(f"❌ 统一配置失败，回退到传统方式: {e}")
    
    # 回退到传统的配置方式
    import openai
    
    # 从IPython存储中检索API_KEY和MODEL_NAME变量
    %store -r API_KEY
    %store -r MODEL_NAME

    # 如果没有设置MODEL_NAME，使用默认值
    try:
        MODEL_NAME
    except NameError:
        MODEL_NAME = "gpt-4o"  # 默认使用gpt-4o模型

    # 创建OpenAI客户端
    client = openai.OpenAI(api_key=API_KEY)

    def get_completion(prompt: str, system_prompt=""):
        """
        获取GPT的完成响应
        
        参数:
            prompt (str): 用户提示
            system_prompt (str): 系统提示（可选）
        
        返回:
            str: GPT的响应文本
        """
        # 构建消息列表
        messages = []
        
        # 如果有系统提示，添加系统消息
        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})
        
        # 添加用户消息
        messages.append({"role": "user", "content": prompt})
        
        # 调用OpenAI API
        response = client.chat.completions.create(
            model=MODEL_NAME,              # 模型名称 (gpt-4o 或 deepseek-r1)
            messages=messages,             # 消息列表
            max_completion_tokens=2000,    # 最大token数
            temperature=0.0               # 温度参数，0表示更确定性
        )
        return response.choices[0].message.content
    
    print("⚠️  使用传统配置方式，建议配置环境变量以获得更好体验")

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
✅ OpenAI环境设置完成!
🔧 OpenAI API 配置信息:
  📡 配置来源: 环境变量 + 自定义API地址: https://vip.apiyi.com/v1
  🤖 模型: gpt-4o
  🌐 API地址: https://vip.apiyi.com/v1
  🔑 API密钥: sk-R2utG...B944

✅ 使用统一配置管理成功！


---

## 📖 理论基础

**给GPT提供您希望它如何表现（或不希望它如何表现）的示例是极其有效的**，这种方法适用于：
- 获得正确的答案
- 以正确的格式获得答案
- 控制输出的语调和风格
- 规范复杂任务的执行流程

### 🔢 样本数量分类

您可能会遇到以下术语，"样本"的数量指的是提示中使用了多少个示例：

| 术语 | 示例数量 | 适用场景 | 优缺点 |
|------|----------|----------|--------|
| **零样本提示** | 0个 | 简单任务、模型已知任务 | ✅ 简洁 ❌ 控制力弱 |
| **单样本提示** | 1个 | 格式示范、简单模式 | ✅ 清晰 ❌ 可能过拟合 |
| **少样本提示** | 2-5个 | 复杂任务、多样化需求 | ✅ 平衡效果 ❌ Token消耗 |
| **多样本提示** | 5+个 | 复杂模式识别 | ✅ 强控制 ❌ 上下文限制 |

### 🎨 示例设计原则

1. **代表性**：示例应该覆盖目标任务的典型情况
2. **多样性**：避免单一模式，展示不同变化
3. **质量**：确保示例本身的质量和准确性
4. **简洁性**：示例要简洁明了，突出关键点

---

## 🚀 实战示例一：客户评价情感分析

让我们通过一个具体例子来理解少样本提示的威力。我们要构建一个客户评价情感分析系统。

### 场景描述
假设我们是一家电商平台，需要自动分析客户评价的情感倾向，并提取关键信息。

### 零样本 vs 少样本对比


In [2]:
# 示例1：零样本提示（效果较差）
zero_shot_prompt = """
请分析以下客户评价的情感倾向，并提取关键信息：

客户评价：物流很快，包装也很好，就是产品质量一般，价格偏贵，不太推荐购买。

请提供情感分析结果。
"""

print("=== 零样本提示 ===")
print(zero_shot_prompt)
print("\n" + "="*50 + "\n")

# 获取零样本结果
zero_shot_result = get_completion(zero_shot_prompt)
print("零样本结果：")
print(zero_shot_result)


=== 零样本提示 ===

请分析以下客户评价的情感倾向，并提取关键信息：

客户评价：物流很快，包装也很好，就是产品质量一般，价格偏贵，不太推荐购买。

请提供情感分析结果。



零样本结果：
情感分析结果：

1. 积极方面：
   - 物流速度快
   - 包装良好

2. 消极方面：
   - 产品质量一般
   - 价格偏贵

总体情感倾向：负面

关键信息提取：
- 客户对物流速度和包装表示满意。
- 客户对产品质量和价格不满意。
- 客户不太推荐购买该产品。


In [3]:
# 示例2：少样本提示（效果显著提升）
few_shot_prompt = """
你是一个专业的客户评价分析师。请按照以下格式分析客户评价：

示例1：
客户评价：这个产品太棒了！质量超出预期，客服态度也很好，五星推荐！
分析结果：
- 整体情感：积极 (9/10)
- 产品质量：非常满意
- 服务体验：非常满意  
- 推荐度：强烈推荐
- 关键词：质量超出预期、客服态度好、五星推荐

示例2：
客户评价：东西收到了，包装还行，但是质量真的一般，感觉不值这个价钱。
分析结果：
- 整体情感：消极 (3/10)
- 产品质量：不满意
- 服务体验：一般
- 推荐度：不推荐
- 关键词：质量一般、不值这个价、包装还行

示例3：
客户评价：物流速度可以，产品功能基本满足需求，但说明书不够详细。
分析结果：
- 整体情感：中性 (6/10)
- 产品质量：基本满意
- 服务体验：满意
- 推荐度：一般
- 关键词：物流速度可以、功能基本满足、说明书不详细

现在请分析：
客户评价：物流很快，包装也很好，就是产品质量一般，价格偏贵，不太推荐购买。
分析结果：
"""

print("=== 少样本提示 ===")
print(few_shot_prompt)
print("\n" + "="*50 + "\n")

# 获取少样本结果
few_shot_result = get_completion(few_shot_prompt)
print("少样本结果：")
print(few_shot_result)


=== 少样本提示 ===

你是一个专业的客户评价分析师。请按照以下格式分析客户评价：

示例1：
客户评价：这个产品太棒了！质量超出预期，客服态度也很好，五星推荐！
分析结果：
- 整体情感：积极 (9/10)
- 产品质量：非常满意
- 服务体验：非常满意  
- 推荐度：强烈推荐
- 关键词：质量超出预期、客服态度好、五星推荐

示例2：
客户评价：东西收到了，包装还行，但是质量真的一般，感觉不值这个价钱。
分析结果：
- 整体情感：消极 (3/10)
- 产品质量：不满意
- 服务体验：一般
- 推荐度：不推荐
- 关键词：质量一般、不值这个价、包装还行

示例3：
客户评价：物流速度可以，产品功能基本满足需求，但说明书不够详细。
分析结果：
- 整体情感：中性 (6/10)
- 产品质量：基本满意
- 服务体验：满意
- 推荐度：一般
- 关键词：物流速度可以、功能基本满足、说明书不详细

现在请分析：
客户评价：物流很快，包装也很好，就是产品质量一般，价格偏贵，不太推荐购买。
分析结果：



少样本结果：
- 整体情感：消极 (4/10)
- 产品质量：不满意
- 服务体验：满意
- 推荐度：不推荐
- 关键词：物流很快、包装很好、质量一般、价格偏贵、不推荐购买


### 📊 效果对比分析

通过上面的对比，我们可以清楚地看到少样本提示的优势：

| 对比维度 | 零样本提示 | 少样本提示 |
|----------|------------|------------|
| **格式一致性** | ❌ 输出格式不固定 | ✅ 严格按照示例格式 |
| **信息完整性** | ❌ 可能遗漏关键信息 | ✅ 全面覆盖各个维度 |
| **分析深度** | ❌ 分析较为浅显 | ✅ 结构化深度分析 |
| **可用性** | ❌ 需要后处理 | ✅ 直接可用 |

### 💡 示例设计技巧

1. **覆盖不同情况**：积极、消极、中性评价
2. **展示期望格式**：结构化的输出格式
3. **提供评分标准**：具体的量化指标
4. **包含边界情况**：复杂的混合情感


---

## 🎭 实战示例二：智能代码注释生成

第二个示例展示如何使用少样本提示来生成高质量的代码注释。这对于代码维护和团队协作非常重要。

### 场景描述
我们需要为Python函数自动生成专业的文档字符串（docstring），包括函数描述、参数说明、返回值和使用示例。


In [4]:
# 少样本提示：代码注释生成
code_documentation_prompt = """
你是一个专业的Python代码文档生成专家。请为给定的函数生成高质量的Google风格docstring。

示例1：
```python
def calculate_bmi(weight, height):
    return weight / (height ** 2)
```

生成文档：
```python
def calculate_bmi(weight, height):
    \"\"\"计算身体质量指数(BMI)。
    
    根据体重和身高计算BMI值，用于评估身体健康状况。
    BMI = 体重(kg) / 身高(m)²
    
    Args:
        weight (float): 体重，单位为千克(kg)，必须为正数
        height (float): 身高，单位为米(m)，必须为正数
        
    Returns:
        float: BMI值，保留小数点后两位
        
    Raises:
        ValueError: 当体重或身高为负数或零时抛出异常
        
    Example:
        >>> calculate_bmi(70.0, 1.75)
        22.86
        >>> calculate_bmi(65, 1.60)
        25.39
    \"\"\"
    return weight / (height ** 2)
```

示例2：
```python
def find_max_in_list(numbers):
    max_num = numbers[0]
    for num in numbers:
        if num > max_num:
            max_num = num
    return max_num
```

生成文档：
```python
def find_max_in_list(numbers):
    \"\"\"在数字列表中查找最大值。
    
    遍历整个列表，找到并返回其中的最大数值。
    该函数使用线性搜索算法，时间复杂度为O(n)。
    
    Args:
        numbers (list): 包含数字的列表，不能为空列表
        
    Returns:
        int/float: 列表中的最大数值，类型与输入元素类型一致
        
    Raises:
        IndexError: 当输入列表为空时抛出异常
        TypeError: 当列表元素不是数字类型时抛出异常
        
    Example:
        >>> find_max_in_list([1, 5, 3, 9, 2])
        9
        >>> find_max_in_list([-1, -5, -2])
        -1
    \"\"\"
    max_num = numbers[0]
    for num in numbers:
        if num > max_num:
            max_num = num
    return max_num
```

现在请为以下函数生成文档：
```python
def merge_sorted_lists(list1, list2):
    result = []
    i, j = 0, 0
    while i < len(list1) and j < len(list2):
        if list1[i] <= list2[j]:
            result.append(list1[i])
            i += 1
        else:
            result.append(list2[j])
            j += 1
    result.extend(list1[i:])
    result.extend(list2[j:])
    return result
```

生成文档：
"""

print("=== 代码文档生成提示 ===")
print("输入函数：")
print("""def merge_sorted_lists(list1, list2):
    result = []
    i, j = 0, 0
    while i < len(list1) and j < len(list2):
        if list1[i] <= list2[j]:
            result.append(list1[i])
            i += 1
        else:
            result.append(list2[j])
            j += 1
    result.extend(list1[i:])
    result.extend(list2[j:])
    return result""")

print("\n" + "="*50 + "\n")

# 获取代码文档生成结果
code_doc_result = get_completion(code_documentation_prompt)
print("生成的文档：")
print(code_doc_result)


=== 代码文档生成提示 ===
输入函数：
def merge_sorted_lists(list1, list2):
    result = []
    i, j = 0, 0
    while i < len(list1) and j < len(list2):
        if list1[i] <= list2[j]:
            result.append(list1[i])
            i += 1
        else:
            result.append(list2[j])
            j += 1
    result.extend(list1[i:])
    result.extend(list2[j:])
    return result


生成的文档：
```python
def merge_sorted_lists(list1, list2):
    """合并两个已排序的列表。

    将两个已排序的列表合并为一个新的排序列表。该函数假设输入的两个列表均已按升序排列。
    使用双指针法进行合并，时间复杂度为O(n + m)，其中n和m分别为两个列表的长度。

    Args:
        list1 (list): 第一个已排序的列表，元素类型为int或float。
        list2 (list): 第二个已排序的列表，元素类型为int或float。

    Returns:
        list: 合并后的排序列表，包含来自两个输入列表的所有元素。

    Raises:
        TypeError: 当列表元素不是数字类型时抛出异常。

    Example:
        >>> merge_sorted_lists([1, 3, 5], [2, 4, 6])
        [1, 2, 3, 4, 5, 6]
        >>> merge_sorted_lists([1, 2, 3], [])
        [1, 2, 3]
        >>> merge_sorted_lists([], [4, 5, 6])
        [4, 5, 6]
    """
    result = []
  

### 2. 🔄 示例质量优化原则

#### 示例选择的黄金法则：

1. **多样性原则**：覆盖不同场景和边界情况
2. **质量原则**：确保示例本身的准确性和专业性  
3. **简洁原则**：示例要简洁明了，突出关键点
4. **相关性原则**：示例与目标任务高度相关
5. **渐进性原则**：从简单到复杂，逐步展示

#### 常见错误避免：

❌ **过度复杂的示例**：示例过于复杂，模型难以抓住重点
❌ **示例不一致**：多个示例之间格式或风格不统一
❌ **缺乏多样性**：所有示例都是同一类型，缺乏覆盖面
❌ **示例有误**：示例本身存在错误，误导模型学习

### 3. 📏 最佳实践总结

| 应用场景 | 推荐示例数量 | 关键技巧 |
|----------|--------------|----------|
| **格式转换** | 2-3个 | 展示输入输出的对应关系 |
| **文本分类** | 3-5个 | 覆盖每个类别的典型示例 |
| **代码生成** | 2-4个 | 从简单到复杂的代码示例 |
| **创意写作** | 3-6个 | 展示不同风格和主题 |
| **数据分析** | 2-3个 | 包含不同数据类型和分析角度 |
